In [1]:
import numpy as np
import random

In [2]:
lines = open('data/berp-POS-training.txt', 'r').readlines()
k = 1.0

In [3]:
### Calculating counts for all

tags = {}
word_tag = {}
tag_tag = {}
prev_tag = '#'
start = {}
end = {}

for line in lines:
    line = line.split()
    if len(line) == 0:
        if tag not in end:
            end[tag] = 0
        end[tag] += 1
        prev_tag = '#'
        continue
        
    word = line[1]
    tag = line[2]
    
    if prev_tag == '#':
        if tag not in start:
            start[tag] = 0
        start[tag] += 1
    else:
        if prev_tag not in tag_tag:
            tag_tag[prev_tag] = {}
        if tag not in tag_tag[prev_tag]:
            tag_tag[prev_tag][tag] = 0
        tag_tag[prev_tag][tag] += 1
        
    if tag not in tags:
        tags[tag] = 0
    tags[tag] += 1
    
    if word not in word_tag:
        word_tag[word] = {}
    if tag not in word_tag[word]:
        word_tag[word][tag] = 0
    word_tag[word][tag] += 1
    
    prev_tag = tag

In [4]:
### Calculating add - k probabilities for all
V = len(tags.keys())
tag_list = list(tags.keys())
N = len(tag_list)
for word in word_tag:
    for tag in tags:
        if tag not in word_tag[word]:
            word_tag[word][tag] = 0
        word_tag[word][tag] = (word_tag[word][tag] + k) / (tags[tag] + (k * V))
        
for prev_tag in tags:
    if prev_tag not in tag_tag:
        tag_tag[prev_tag] = {}
    for tag in tags:
        if tag not in tag_tag[prev_tag]:
            tag_tag[prev_tag][tag] = 0
        tag_tag[prev_tag][tag] = (tag_tag[prev_tag][tag] + k) / (tags[prev_tag] + (k * V))
        
den = sum(list(start.values())) + (k * V)
for tag in tags:
    if tag not in start:
        start[tag] = 0
    start[tag] = (start[tag] + k) / den
    
den = sum(list(end.values())) + (k * V)
for tag in tags:
    if tag not in end:
        end[tag] = 0
    end[tag] = (end[tag] + k) / den

In [5]:
test_lines = open('data/test-set.txt', 'r').readlines()
lines = []

In [6]:
string = ""
for line in test_lines:
    line = line.split()
    
    if len(line) == 0:
        lines.append(string)
        string = ""
        continue
    
    string += line[1] + ' '

In [7]:
pred_lines = []
N = len(tag_list)

In [8]:
for line in lines:
    T = len(line.split())
    
    line = line.split()
    viterbi = np.zeros((N, T))
    backpointer = np.zeros((N, T))
    
    # initialization step
    word = line[0]
    for s in range(len(tag_list)):
        start_prob = start[tag_list[s]]
        
        try:
            emission = word_tag[word][tag_list[s]]
        
        except:
            word = random.choice(list(word_tag.keys()))
            tag = random.choice(list(word_tag[word].keys()))
            emission = word_tag[word][tag]

        viterbi[s][0] = start_prob * emission
        backpointer[s][0] = 0
            
    # recursion step
    for t in range(1, T):
        for s in range(len(tag_list)):
            _max = []
            try:
                emission = word_tag[line[t]][tag_list[s]]
            except:
                word = random.choice(list(word_tag.keys()))
                tag = random.choice(list(word_tag[word].keys()))
                emission = word_tag[word][tag]
            for _s in range(len(tag_list)):
                transition = tag_tag[tag_list[_s]][tag_list[s]]
                prod = emission*transition*viterbi[_s][t-1]
                _max.append(prod)
                
            viterbi[s][t] = max(_max)
            backpointer[s][t] = _max.index(max(_max))
            
    # termination step
    _max = []
    for s in range(len(tag_list)):
        _max.append(viterbi[s][-1] * end[tag_list[s]])
    
    end_path = _max.index(max(_max))

    # backtracking        
    pred_tags = []
    pred_tags.append((line[T-1], tag_list[end_path]))
    for t in range(T-1, 0, -1):
        pred_tags.append((line[t-1], tag_list[int(backpointer[end_path][t])]))
        end_path = int(backpointer[end_path][t])
    pred_lines.append(pred_tags[::-1])


In [9]:
f = open('data/my-output.txt', 'w')
for line in pred_lines:
    i = 1
    for word_tags in line:
        f.write(str(i) + '\t' + word_tags[0] + '\t' + word_tags[1] + '\n')
        i += 1
    f.write('\n')
f.close()